# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [56]:
def priest(sex, age, rspRt, oxSt, hrtRt, sybp, tmp, alrtns, inspOyg, sts):
    """
    (String, integer, integer, integer, integer, integer, float, String, String, String) -> float)

    Priest function is calculated using risk percentage as a predictor of
    undesirable effects in adults who have presented to ED with presumed COVID-19.

    Priest function expects sex, alrtns, inspOxy and sts in String and remaining all are in float.

    >>> priest("male",65,10,0.96,135,91,38.1,"ALERT","AIR","unrestricted normal activity")
    0.29

    >>> priest("Female",18,10,0.96,90,230,38.1,"ALERT","AIR","limited self-care")
    0.22

    >>> priest("Female",40,8,0.96,38,115,36.3,"ALERT","AIR","limited strenuous activity, can do light activity")
    0.18

    """

    sex = sex.lower()
    alrtns = alrtns.lower()
    inspOyg = inspOyg.lower()
    sts = sts.lower()
    count = 0

    if sex=="male":
        count += 1

    if age >= 50 and age<=65:
        count = count + 2
    elif age >= 66 and age<=80:
        count = count + 3
    elif age > 80:
        count = count + 4

    if rspRt > 24:
        count = count +  3
    elif rspRt >= 21 and rspRt <= 24:
        count = count +  2
    elif rspRt >= 9 and rspRt <= 11:
        count = count +  1
    elif rspRt < 9:
        count = count +  3

    if oxSt < 0.92:
        count = count +  3
    elif oxSt >= 0.94 and oxSt <= 0.95:
        count = count +  1
    elif oxSt >= 0.92 and oxSt <= 0.93:
        count = count +  2

    if hrtRt > 130:
        count = count +  3
    elif hrtRt >= 111 and hrtRt <= 130:
        count = count +  2
    elif hrtRt >= 91 and hrtRt <= 110:
        count = count +  1
    elif hrtRt < 41:
        count = count +  3
    elif hrtRt >= 41 and hrtRt <= 50:
        count = count +  1

    if sybp > 219:
        count = count +  3
    elif sybp >= 101 and sybp <= 110:
        count = count +  1
    elif sybp >= 91 and sybp <= 100:
        count = count +  2
    elif sybp < 91:
        count = count +  3

    if tmp > 39.0:
        count = count +  2
    elif tmp >= 38.1 and tmp <= 39.0:
        count = count +  1
    if tmp < 35.1:
        count = count +  3
    elif tmp >= 35.1 and tmp <= 36.0:
        count = count +  1

    if alrtns == "confused or not alert":
        count = count +  3

    if inspOyg == "supplemental oxygen":
        count = count +  2

    if sts == "limited self-care":
        count = count +  3
    elif sts == "limited activity, can self-care":
        count = count +  2
    if sts == "limited strenuous activity, can do light activity":
        count = count +  1
    elif sts == "bed/chair bound, no self-care":
        count = count +  4

    riskPtg = [0.01, 0.01, 0.02, 0.02, 0.03, 0.09, 0.15, 0.18, 0.22, 0.26, 0.29, 0.34, 0.38, 0.46, 0.47, 0.49, 0.55, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,0.99]
    count = min(count, len(riskPtg) - 1)
    return riskPtg[count]

In [57]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest("male",65,10,0.96,135,91,38.1,"ALERT","AIR","unrestricted normal activity")
Expecting:
    0.29
ok
Trying:
    priest("Female",18,10,0.96,90,230,38.1,"ALERT","AIR","limited self-care")
Expecting:
    0.22
ok
Trying:
    priest("Female",40,8,0.96,38,115,36.3,"ALERT","AIR","limited strenuous activity, can do light activity")
Expecting:
    0.18
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [58]:
import json
import requests
import csv


def find_hospital(age, sex, riskPerc):
    """
    (integer, String, float) -> String

    Find_hospital function Use the age, sex and risk percentage for finding a hospital. It
    expects age, risk percentage in float and sex in String and produces hospital name in String.

     find_hospital function find hospital on API by passing age, sex and rper as query parameters to the url
     using requests module


    >>> find_hospital(66, "female", 0.18)
    'Wesley Woods Geriatric Hospital'

    >>> find_hospital(18, "Male", 0.26)
    'Southwest Hospital and Medical Center'

    >>> find_hospital(40, "FEMALE", 0.09)
    'Select Specialty Hospital - Northeast Atlanta'

    """
    sex = sex.lower()
    apiurl = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/"
    apidata = requests.get(apiurl, params={"age": age, "sex": sex, "risk_pct": riskPerc})
    return (apidata.json())['hospital']

print(find_hospital(18, "Male", 0.26))

Southwest Hospital and Medical Center


In [59]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(66, "female", 0.18)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok
Trying:
    find_hospital(18, "Male", 0.26)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(40, "FEMALE", 0.09)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [60]:
def get_address(hospital):

    """
    (String) -> String

    get_address function Find a hospital's address on the basis of its hospital name from this api data

    get_address function determines the hospital's name as String and returns address in String


    >>> get_address("Wesley Woods Geriatric Hospital")
    '1821 CLIFTON ROAD, NE'

    >>> get_address("Emory Dunwoody Medical Center")
    '4500 NORTH SHALLOWFORD ROAD'

    >>> get_address("Select Specialty Hospital - Northeast Atlanta")
    '1821 CLIFTON ROAD NE'

    """

    apiurl = "https://hds5210-data.s3.amazonaws.com/ga_hospitals.json"
    apidata = requests.get(apiurl)
    hospital=hospital.upper()
    if hospital in (apidata.json()):
        return (apidata.json())[hospital]["ADDRESS"]
    else:
        return " "

## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [61]:
import json
import csv
import requests

def process_people(apiurl):
    """
    (String) -> Dictionary

    process_people function is determining the risk percentage, hospital to be visited and address From the group of patients.

    Process_people function accepts floc as a string and returns dictionary in json format. And also It gathers data in the
    csv file from the API and computes risk percentage, hospital to visit and address of the hospital and append
    each patient with information including all factors and risk percentage,
    The hospital's name and address in json format for the dictionary.

    """

    apidata = requests.get(apiurl)
    content = apidata.text.split("\n")
    fd = csv.reader(content, delimiter='|')
    next(fd)
    emptydict = {}
    for i in fd:
        if not any(field != '' for field in i):
            break
        patient, sex, age, rspRt, oxSt, hrtRt, sybp, tmp, alrtns, inspOyg, sts = i
        riskperc = priest(sex, float(age), float(rspRt), float(oxSt), float(hrtRt), float(sybp), float(tmp), alrtns, inspOyg, sts)
        hospital = find_hospital(int(age), sex, riskperc)
        addressHosp = get_address(hospital)
        data = [sex, float(age), float(rspRt), float(oxSt), float(hrtRt), float(sybp), float(tmp), alrtns, inspOyg, sts, riskperc, hospital, addressHosp]
        emptydict[patient] = data

    return emptydict

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [62]:
"""

 It's comparing the actual API data to processed data.
 From the above methods, you will see whether either of them is equal or not.

"""

apiurl = "https://hds5210-data.s3.amazonaws.com/people_results.json"
processedData = process_people("https://hds5210-data.s3.amazonaws.com/people.psv")
apidata = requests.get(apiurl)
APIdata = apidata.json()
X = json.loads(json.dumps(APIdata))
Y= json.loads(json.dumps(processedData))
if  X == Y:
    print("Actual API Data is same as Processed Data")
else:
    print("They are not Equal")

Actual API Data is same as Processed Data


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---